In [1]:
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error

data = pd.read_csv("data.csv")

data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [10]:
LOOK_BACK = 30
PREDICT_DAY = 1

In [3]:
def create_dataset(dataset, look_back=LOOK_BACK):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:i+look_back, :])
        Y.append(dataset[i+look_back, :])
    return np.array(X), np.array(Y)

X, Y = create_dataset(scaled_data)
train_size = int(len(X) * 0.8)
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]



In [4]:
model = Sequential()

# Layer 1
model.add(
    SimpleRNN(
        50,
        input_shape=(LOOK_BACK, data.shape[1]),
        return_sequences=True,
        activation="relu",
    )
)

# Layer 2
model.add(SimpleRNN(50, return_sequences=True, activation="relu"))
model.add(Dropout(0.2))

# Layer 3
model.add(SimpleRNN(50, return_sequences=True, activation="relu"))
model.add(Dropout(0.2))

# Layer 4
model.add(SimpleRNN(50, return_sequences=True, activation="relu"))
model.add(Dropout(0.2))

# Layer 5
model.add(SimpleRNN(50, activation="relu"))

model.add(Dense(data.shape[1]))

model.compile(loss="mean_squared_error", optimizer="adam")

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 30, 50)            3100      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 30, 50)            5050      
                                                                 
 dropout (Dropout)           (None, 30, 50)            0         
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 30, 50)            5050      
                                                                 
 dropout_1 (Dropout)         (None, 30, 50)            0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 30, 50)            5050      
                                                                 
 dropout_2 (Dropout)         (None, 30, 50)            0

In [6]:
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=1)

Epoch 1/10
2894/2894 [==============================] - 41s 13ms/step - loss: 0.0538
Epoch 2/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0294
Epoch 3/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0247
Epoch 4/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0244
Epoch 5/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0219
Epoch 6/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0219
Epoch 7/10
2894/2894 [==============================] - 38s 13ms/step - loss: 0.0223
Epoch 8/10
2894/2894 [==============================] - 38s 13ms/step - loss: 0.0217
Epoch 9/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0233
Epoch 10/10
2894/2894 [==============================] - 37s 13ms/step - loss: 0.0218


In [7]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY_original = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY_original = scaler.inverse_transform(testY)



23/23 [==============================] - 0s 5ms/step


In [8]:

testPredictions = []
for i in range(len(testX)):
    prediction = model.predict(testX[i:i+1])
    testPredictions.append(prediction[0])

testPredictions = np.array(testPredictions)

testPredictions = scaler.inverse_transform(testPredictions)
testY_original = scaler.inverse_transform(testY)

1/1 [==============================] - 0s 22ms/step


In [11]:
random_start_index = random.randint(0, len(X) - PREDICT_DAY)


random_input_sequence = X[random_start_index]
true_output = Y[random_start_index]


random_input_reshaped = np.reshape(random_input_sequence, (1, LOOK_BACK, data.shape[1]))

predicted_output = model.predict(random_input_reshaped)


predicted_output = scaler.inverse_transform(predicted_output)
true_output_original = scaler.inverse_transform([true_output])


MSE = mean_squared_error(true_output_original, predicted_output)
MAE = mean_absolute_error(true_output_original, predicted_output)
MAPE = np.mean(np.abs((true_output_original - predicted_output) / true_output_original)) * 100

print('Random Sequence Evaluation:')
print('MSE:', MSE)
print('MAE:', MAE)
print('MAPE:', MAPE)

1/1 [==============================] - 0s 14ms/step
Random Sequence Evaluation:
MSE: 3753.863365517784
MAE: 18.7557585337032
MAPE: 8.826273091569515
